In [1]:
!pip install pandas

    100% |████████████████████████████████| 10.1MB 1.3MB/s eta 0:00:01
    100% |████████████████████████████████| 512kB 18.0MB/s ta 0:00:01
    100% |████████████████████████████████| 17.3MB 907kB/s eta 0:00:01


In [2]:
import pandas as pd
#import numpy as np
#import requests # library to handle requests
#import json # library to handle JSON files
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.6.13

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /srv/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.19.0               |             py_0          53 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          85 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.19.0-py_0 conda-forge


geographiclib-1.49   | 32 KB     | ##################################### | 100% 
geopy-1.19.0         | 53 KB     | ########################

In [6]:
min_date = "1+February+2019"
csv_link = "http://landregistry.data.gov.uk/app/ppd/ppd_data.csv?et%5B%5D=lrcommon%3Afreehold&et%5B%5D=lrcommon%3Aleasehold&header=true&limit=all&min_date={}&nb%5B%5D=true&nb%5B%5D=false&ptype%5B%5D=lrcommon%3Adetached&ptype%5B%5D=lrcommon%3Asemi-detached&ptype%5B%5D=lrcommon%3Aterraced&ptype%5B%5D=lrcommon%3Aflat-maisonette&ptype%5B%5D=lrcommon%3AotherPropertyType&tc%5B%5D=ppd%3AstandardPricePaidTransaction&tc%5B%5D=ppd%3AadditionalPricePaidTransaction".format(min_date)
df = pd.read_csv(csv_link, header=None)
#df = pd.read_csv("http://landregistry.data.gov.uk/app/ppd/ppd_data.csv?et%5B%5D=lrcommon%3Afreehold&et%5B%5D=lrcommon%3Aleasehold&header=true&limit=all&min_date=1+February+2019&nb%5B%5D=true&nb%5B%5D=false&ptype%5B%5D=lrcommon%3Adetached&ptype%5B%5D=lrcommon%3Asemi-detached&ptype%5B%5D=lrcommon%3Aterraced&ptype%5B%5D=lrcommon%3Aflat-maisonette&ptype%5B%5D=lrcommon%3AotherPropertyType&tc%5B%5D=ppd%3AstandardPricePaidTransaction&tc%5B%5D=ppd%3AadditionalPricePaidTransaction",  header=None)

#Update column headers
headers = ['unique_id','price_paid','deed_date','postcode','property_type','new_build','estate_type','saon','paon','street','locality','town','district','county','transaction_category','linked_data_uri']
df.columns = headers

print(df.shape)

(23674, 16)


In [7]:
#remove 'additional' transaction category
df = df[df['transaction_category']=='A']

df = df[df['property_type']=='F']

df = df[df['new_build']=='N']

df = df[df['county']=='GREATER LONDON']

df.drop(columns=['unique_id','deed_date', 'property_type', 'new_build','estate_type','saon','paon','locality','district','transaction_category','linked_data_uri'], inplace=True)

print(df.shape)
df.head()

,price_paid,postcode,street,town,county
2334,240000,BR1 3AG,FREELANDS ROAD,BROMLEY,GREATER LONDON
2335,315000,BR1 3AX,WIDMORE ROAD,BROMLEY,GREATER LONDON
2342,280000,BR1 3TY,OAKLANDS ROAD,BROMLEY,GREATER LONDON
2353,200000,BR2 0AB,BROMLEY ROAD,BROMLEY,GREATER LONDON
2354,280000,BR2 0JW,BECKENHAM GROVE,BROMLEY,GREATER LONDON


In [8]:
#Add lontitude and lattitude data
addresses = df['postcode']
latitude = []
longitude = []

for address in addresses:
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address+", UK", timeout=2)
    try:
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    except:
        latitude.append(0)
        longitude.append(0)
        
df['Latitude']= latitude
df['Longitude']= longitude

In [9]:
df.head()


,price_paid,postcode,street,town,county,Latitude,Longitude
2334,240000,BR1 3AG,FREELANDS ROAD,BROMLEY,GREATER LONDON,51.408158,0.024103
2335,315000,BR1 3AX,WIDMORE ROAD,BROMLEY,GREATER LONDON,51.406592,0.030616
2342,280000,BR1 3TY,OAKLANDS ROAD,BROMLEY,GREATER LONDON,51.414684,0.003351
2353,200000,BR2 0AB,BROMLEY ROAD,BROMLEY,GREATER LONDON,51.407010,-0.004864
2354,280000,BR2 0JW,BECKENHAM GROVE,BROMLEY,GREATER LONDON,51.403289,-0.002394


In [10]:
#remove regions that location information was not available
df = df[(df['Latitude']!=0)| (df['Longitude']!=0)]
print(df.shape[0])
df.head()

779


,price_paid,postcode,street,town,county,Latitude,Longitude
2334,240000,BR1 3AG,FREELANDS ROAD,BROMLEY,GREATER LONDON,51.408158,0.024103
2335,315000,BR1 3AX,WIDMORE ROAD,BROMLEY,GREATER LONDON,51.406592,0.030616
2342,280000,BR1 3TY,OAKLANDS ROAD,BROMLEY,GREATER LONDON,51.414684,0.003351
2353,200000,BR2 0AB,BROMLEY ROAD,BROMLEY,GREATER LONDON,51.407010,-0.004864
2354,280000,BR2 0JW,BECKENHAM GROVE,BROMLEY,GREATER LONDON,51.403289,-0.002394


In [ ]:
CLIENT_ID = 'QKP4KVQQZPVXMOKNIXHGQAT1E2IJZCHQPFB4XJGP3MC0CFUO' # your Foursquare ID
CLIENT_SECRET = '1N5BIRBR1EA5CO1QFMNHU1241SOFM5KJLLQRJGMDC0NOYWPH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 200 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
categoryId = ''
#categoryId = '4bf58dd8d48988d103951735'

In [ ]:
#What are the most popular places nearby?

def getNearbyVenues(names, latitudes, longitudes, avg_price, radius=500):
   
    region_list = []
    price_list = []
    
    for name, lat, lng, price in zip(names, latitudes, longitudes, avg_price):
                    
        # create the API request URL
        #url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        #url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(    
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryId)
        
        #print(url) 
        
        # make the GET request
        results = requests.get(url).json()["response"]
        
        #flatten JSON file
        resultsVen = json_normalize(results['venues'])
        resultsCat = json_normalize(data=results['venues'], record_path=['categories'], record_prefix='cat_', errors='ignore')
        resultsVenCat = pd.concat([resultsVen, resultsCat], axis=1)
        resultsVenCat = resultsVenCat[['id','name','location.distance','location.formattedAddress','location.lat','location.lng','cat_name']]
        
        #Add region name and average price 
        for i in range(0,resultsVen.shape[0]):
            region_list.append(name)
            price_list.append(price)
        
        region_list = pd.DataFrame(region_list, columns=['Region Name'])
        price_list = pd.DataFrame(price_list, columns=['Region Avg Price'])
        
        nearbyVenues = pd.concat([region_list, price_list, resultsVenCat], axis=1)

    return(nearbyVenues)

In [ ]:
regions_venues = pd.DataFrame()

#request venue information for each region and store in regions_venues
for code in df['postcode'].unique():
    region_venues = df[df['postcode']==code]
    regions_venues = regions_venues.append((getNearbyVenues(names=region_venues['postcode'],
                                                  latitudes=region_venues['Latitude'],
                                                  longitudes=region_venues['Longitude'],
                                                  avg_price=region_venues['price_paid'])),ignore_index=True)
    
#rename columns
regions_venues.rename(columns={'id':'Venue ID',
                          'name':'Venue Name',
                          'location.distance':'Venue Distance',
                          'location.formattedAddress':'Venue Address',
                          'location.lat':'Venue Lat',
                          'location.lng':'Venue Lng',
                          'cat_name':'Venue Category'}, inplace=True)

regions_venues.head()